# **STIB-MIVB Vehicle Position Preprocessing & DB Load**

In [1]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import psycopg2
import psycopg2.extras as extras
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from json_normalize import json_normalize
from IPython.display import display, HTML, display_html

## 1. Load & Preprocess Shapefile Data

In [2]:
# load lines & stops from shapefile, as GeoDataFrame (gdf)

stib_lines = gpd.read_file("stib_data/network/ACTU_LINES.shp")
stib_stops = gpd.read_file("stib_data/network/ACTU_STOPS.shp")

In [3]:
# line gdf attributes

stib_lines.head(3)

,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,geometry
0,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,001m,2,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (156746.700 170167.000 0.000, 156..."
2,002m,1,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147305.500 172526.900 0.000, 147..."


In [4]:
# stop gdf attributes

stib_stops.head(3)

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry
0,012b,1,1,9600B,BRUSSELS AIRPORT,BRUSSELS AIRPORT,Brussels Airport,Brussels Airport,157950.0,176429.0,B,12,BRUSSELS CITY,POINT (157950.000 176429.000)
1,012b,1,2,3017,BOURGET,BOURGET,Bourget,Bourget,154334.0,174200.0,B,12,BRUSSELS CITY,POINT (154334.000 174200.000)
2,012b,1,3,5048,DA VINCI,DA VINCI,Da Vinci,Da Vinci,152934.0,173976.0,B,12,BRUSSELS CITY,POINT (152934.000 173976.000)


In [5]:
# check for null values

print(f'Null Values in Lines GDF: {stib_lines.isna().sum().max()}')
print(f'Null Values in Stops GDF: {stib_stops.isna().sum().max()}')

Null Values in Lines GDF: 0
Null Values in Stops GDF: 0


In [6]:
# clean lines gdf

stib_lines = stib_lines.rename(
    str.lower, 
    axis = 'columns'
).rename(
    columns = {
        'ligne': 'line_code', 
        'variante': 'direction'
    }
)

In [7]:
# clean stops gdf

stib_stops = stib_stops.rename(
    str.lower, 
    axis = 'columns'
).drop(
    columns = ['descr_nl', 'alpha_fr', 'alpha_nl'], 
    axis = 1
).rename(
    columns = {
        'code_ligne': 'line_code', 
        'variante': 'direction',
        'succession': 'stop_seq',
        'descr_fr': 'stop_name',
        'mode': 'vehicle',
        'numero_lig': 'line',
        'terminus': 'end_stop'
    }
)

# map vehicle names for better readability

for index, row in stib_stops.iterrows():
    if row['vehicle'] == 'B':
        stib_stops.at[index,'vehicle'] ='BUS'
    elif row['vehicle'] == 'T':
        stib_stops.at[index,'vehicle'] = 'TRAM'
    elif row['vehicle'] == 'M':
        stib_stops.at[index,'vehicle'] = 'METRO'
    else:
        pass

In [8]:
# get distinct stops & lines

distinct_stop_lines = stib_stops[['line_code', 'vehicle', 'line']].drop_duplicates()
distinct_lines = stib_lines[['line_code', 'color_hex']].drop_duplicates()

In [9]:
# total number of line types by vehicle

distinct_stop_lines.vehicle.value_counts()

BUS      65
TRAM     18
METRO     4
Name: vehicle, dtype: int64

In [10]:
# distinct stop stations

print(f'Total distinct stop stations: {len(stib_stops.stop_name.drop_duplicates())}')

Total distinct stop stations: 894


In [11]:
# get additional attributes for lines gdf

stib_lines = stib_lines.merge(
    distinct_stop_lines, 
    on = 'line_code', how = 'inner'
)[[
    'vehicle', 
    'line', 
    'direction', 
    'date_debut', 
    'date_fin', 
    'color_hex', 
    'geometry'
]]

# get additional attributes for stops gdf

stib_stops = stib_stops.merge(
    distinct_lines, 
    on = 'line_code', how = 'inner'
)[[
    'vehicle', 
    'line', 
    'direction', 
    'stop_seq', 
    'stop_id', 
    'stop_name',
    'color_hex',
    'end_stop',
    'geometry'
]]

In [12]:
# lines gdf attributes (after initial preprocessing)

stib_lines.head(3)

,vehicle,line,direction,date_debut,date_fin,color_hex,geometry
0,METRO,1,1,01/09/2021,06/03/2022,#C4008F,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,METRO,1,2,01/09/2021,06/03/2022,#C4008F,"LINESTRING Z (156746.700 170167.000 0.000, 156..."
2,METRO,2,1,01/09/2021,06/03/2022,#F57000,"LINESTRING Z (147305.500 172526.900 0.000, 147..."


In [13]:
# stops gdf attributes (after initial preprocessing)

stib_stops.head(3)

,vehicle,line,direction,stop_seq,stop_id,stop_name,color_hex,end_stop,geometry
0,BUS,12,1,1,9600B,BRUSSELS AIRPORT,#338C26,BRUSSELS CITY,POINT (157950.000 176429.000)
1,BUS,12,1,2,3017,BOURGET,#338C26,BRUSSELS CITY,POINT (154334.000 174200.000)
2,BUS,12,1,3,5048,DA VINCI,#338C26,BRUSSELS CITY,POINT (152934.000 173976.000)


In [14]:
# remove any duplicates and strip off letters from stop_id

shp_stops = stib_stops[['vehicle', 'line', 'direction', 'stop_seq', 'stop_id', 'stop_name', 'end_stop']].drop_duplicates()
shp_stops['stop_id'] = shp_stops['stop_id'].str.extract('(\d+)', expand = False).astype('int64')

shp_stops.head()

,vehicle,line,direction,stop_seq,stop_id,stop_name,end_stop
0,BUS,12,1,1,9600,BRUSSELS AIRPORT,BRUSSELS CITY
1,BUS,12,1,2,3017,BOURGET,BRUSSELS CITY
2,BUS,12,1,3,5048,DA VINCI,BRUSSELS CITY
3,BUS,12,1,4,2695,GENEVE,BRUSSELS CITY
4,BUS,12,1,5,2250,MEISER,BRUSSELS CITY


In [15]:
# keep stop names to upper, fix the line names for night schedules

shp_stops['stop_name'] = shp_stops['stop_name'].str.upper()
shp_stops['end_stop'] = shp_stops['end_stop'].str.upper()

shp_stops['line'] = shp_stops['line'].apply(
    lambda x: 'N' + str(x)[1:] if len(str(x)) == 3 else x
).astype('str')

shp_stops.head()

,vehicle,line,direction,stop_seq,stop_id,stop_name,end_stop
0,BUS,12,1,1,9600,BRUSSELS AIRPORT,BRUSSELS CITY
1,BUS,12,1,2,3017,BOURGET,BRUSSELS CITY
2,BUS,12,1,3,5048,DA VINCI,BRUSSELS CITY
3,BUS,12,1,4,2695,GENEVE,BRUSSELS CITY
4,BUS,12,1,5,2250,MEISER,BRUSSELS CITY


## 2. Load GTFS Data (Preprocessed)

In [16]:
# set up connection variables
db_host = "localhost"
db_port = "5432"
db_user = "postgres"
db_pass = "password"
db_name = "stib_transport"

# function to connect with postgres
def connect_postgres(db_host, db_port, db_user, db_pass, db_name):
    try:
        # Connect to an existing database
        connection = psycopg2.connect(host = db_host,
                                      port = db_port,
                                      user = db_user,
                                      password = db_pass,
                                      database = db_name)
        # Set auto-commit
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
        # Create a cursor to perform database operations
        cur = connection.cursor()
        # Print PostgreSQL details
        print("PostgreSQL server information")
        print(connection.get_dsn_parameters(), "\n")
        # Executing a SQL query
        cur.execute("SELECT version();")
        # Fetch result
        record = cur.fetchone()
        print("You are connected to - ", record, "\n")

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    else:
        return cur

In [17]:
# connect to postgres

cur = connect_postgres(db_host, db_port, db_user, db_pass, db_name)

PostgreSQL server information
{'user': 'postgres', 'dbname': 'stib_transport', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.5, compiled by Visual C++ build 1914, 64-bit',) 



In [18]:
# query to get all the stops from gtfs schedule

cur.execute(
    """
    
    -- get all stops from gtfs schedule
    
    with base as (
        select
          distinct
          r.vehicle
        , r.line
        , t.direction
        , st.stop_id
        , upper(s.stop_name) as stop_name
        from sched_trips t
        inner join sched_routes r
            on t.route_id = r.route_id
        inner join sched_calendar c
            on t.service_id = c.service_id
        inner join sched_stop_times st
            on t.trip_id = st.trip_id
        inner join sched_stops s
            on st.stop_id = s.stop_id
    )

    select 
    * 
    from base;
        
    """
)

gtfs_stops = pd.DataFrame(cur.fetchall(), columns = [desc[0] for desc in cur.description])

In [19]:
# strip off the letters from stop_id

gtfs_stops['stop_id'] = gtfs_stops['stop_id'].str.extract('(\d+)', expand = False).astype('int64')

gtfs_stops.head()

,vehicle,line,direction,stop_id,stop_name
0,BUS,T7,1,3508,FLAGEY
1,BUS,38,2,1969,HOUZEAU
2,METRO,6,1,8472,ELISABETH
3,BUS,27,1,1140,DEGROOFF
4,TRAM,3,2,6179,MASUI


## 3. Combine Shapefile & GTFS Data

In [20]:
# left join the shapefile stops with gtfs stops

stop_details = shp_stops.merge(
    gtfs_stops,
    how = 'left',
    left_on = ['vehicle', 'line', 'direction', 'stop_name'],
    right_on = ['vehicle', 'line', 'direction', 'stop_name']
)

stop_details.stop_id_x = stop_details.stop_id_x.astype('float64')

In [21]:
# example of case where stop_id 8162 for Stockel was retreived from gtfs as it was not in shapefile

stop_details.loc[stop_details.stop_name == 'STOCKEL'][['stop_id_x', 'stop_id_y']].drop_duplicates()

,stop_id_x,stop_id_y
508,1252.0,1252.0
515,1302.0,1302.0
3246,8161.0,8161.0
3247,8161.0,8162.0
3908,6474.0,6474.0
3925,6475.0,6475.0


In [22]:
# remap the stop_id to add the missing ones

for index, row in stop_details.iterrows():
    if (row['stop_id_x'] != row['stop_id_y']) & ~(pd.isna(row['stop_id_y'])):
        stop_details.at[index,'stop_id_x'] = row['stop_id_y']
    else:
        pass

In [23]:
# clean up the dataframe

stop_details = stop_details.drop(
    columns = ['stop_id_y']
).rename(
    columns = {
        'stop_id_x': 'stop_id'
    }
)

stop_details.stop_id = stop_details.stop_id.astype('int64')

In [24]:
# join the stop details with itself, to get end_stop_id for each row

stop_details = stop_details.merge(
    stop_details, 
    how = 'inner', 
    left_on = ['vehicle', 'line', 'end_stop'], 
    right_on = ['vehicle', 'line', 'stop_name'],
)[['vehicle', 'line', 'direction_x', 'stop_seq_x', 'stop_id_x', 'stop_name_x', 'stop_id_y', 'end_stop_x']].drop_duplicates().rename(
    columns = {
        'direction_x': 'direction',
        'stop_seq_x': 'stop_seq',
        'stop_id_x': 'stop_id',
        'stop_name_x': 'stop_name',
        'stop_id_y': 'end_stop_id',
        'end_stop_x': 'end_stop_name'
    }
).drop_duplicates(
    subset = ['line', 'direction', 'stop_id', 'stop_name', 'end_stop_id', 'end_stop_name']
).reset_index(drop = True)

stop_details.head()

,vehicle,line,direction,stop_seq,stop_id,stop_name,end_stop_id,end_stop_name
0,BUS,12,2,1,1780,TRONE,9600,BRUSSELS AIRPORT
1,BUS,12,2,1,6433,TRONE,9600,BRUSSELS AIRPORT
2,BUS,12,2,3,1131,LUXEMBOURG,9600,BRUSSELS AIRPORT
3,BUS,12,2,4,1418,SCHUMAN,9600,BRUSSELS AIRPORT
4,BUS,12,2,4,1318,SCHUMAN,9600,BRUSSELS AIRPORT


## 4. Load & Preprocess JSON Data

In [25]:
# generate list for targeting each json file

file_index = []

for i in np.arange(1, 14):
    file_index.append(str("{0:0=2d}".format(int(i))))
    
file_index

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13']

In [26]:
# load all the json files into one list

j_data = []

for i in range(len(file_index)):
    with open(f'stib_data/vehicle_position/vehiclePosition{file_index[i]}.json') as data_file:    
        j_data.append(json.load(data_file))
        data_file.close()

In [27]:
# convert json data to tabular and insert into dataframe

normal_json = json_normalize(j_data[0], key_joiner = "_")
vh_pos = pd.DataFrame(normal_json)
print(f'DataFrame size of vh_pos after inserting file 01: {len(vh_pos)}\n')


for i in range(1, len(file_index)):
    normal_json = json_normalize(j_data[i], key_joiner = "_")
    df = pd.DataFrame(normal_json)
    vh_pos = pd.concat([vh_pos, df])
    print(f'DataFrame size of vh_pos after inserting file {file_index[i]}: {len(vh_pos)}\n')

DataFrame size of vh_pos after inserting file 01: 1365664

DataFrame size of vh_pos after inserting file 02: 4269802

DataFrame size of vh_pos after inserting file 03: 4682622

DataFrame size of vh_pos after inserting file 04: 5611699

DataFrame size of vh_pos after inserting file 05: 6284508

DataFrame size of vh_pos after inserting file 06: 8879437

DataFrame size of vh_pos after inserting file 07: 9995351

DataFrame size of vh_pos after inserting file 08: 10141959

DataFrame size of vh_pos after inserting file 09: 12805101

DataFrame size of vh_pos after inserting file 10: 16354207

DataFrame size of vh_pos after inserting file 11: 17466632

DataFrame size of vh_pos after inserting file 12: 19139610

DataFrame size of vh_pos after inserting file 13: 19423626



In [28]:
# clean up the dataframe

vh_pos = vh_pos.rename(
    columns = {
        'data_time': 'vh_timestamp',
        'data_Responses_lines_lineId': 'line',
        'data_Responses_lines_vehiclePositions_directionId': 'end_stop_id',
        'data_Responses_lines_vehiclePositions_distanceFromPoint': 'distance_from_prev_stop',
        'data_Responses_lines_vehiclePositions_pointId': 'prev_stop_id'
    },
).drop(columns = 'data_Responses')

vh_pos['vh_timestamp'] = pd.to_datetime(vh_pos['vh_timestamp'], unit = 'ms')
vh_pos = vh_pos.loc[~(vh_pos.line.isna())].reset_index(drop = True)
vh_pos.end_stop_id = vh_pos.end_stop_id.astype('int64')
vh_pos.prev_stop_id = vh_pos.prev_stop_id.astype('int64')

vh_pos.head()

,vh_timestamp,line,end_stop_id,distance_from_prev_stop,prev_stop_id
0,2021-09-06 07:54:46.924,1,8161,1.0,8012
1,2021-09-06 07:54:46.924,1,8162,0.0,8142
2,2021-09-06 07:54:46.924,1,8162,0.0,8282
3,2021-09-06 07:54:46.924,1,8731,0.0,8111
4,2021-09-06 07:54:46.924,1,8162,1.0,8062


## 5. Combine JSON Data with Shapefile & GTFS Data (First Mapping)

In [29]:
# left join vehicle position data (json) with the shapefile+gtfs data to get additional attributes that are essential

vh_pos_first_mapping = vh_pos.merge(
    stop_details,
    how = 'left',
    left_on = ['line', 'end_stop_id', 'prev_stop_id'],
    right_on = ['line', 'end_stop_id', 'stop_id']
)

In [30]:
# get both the matched and mismatched data

json_matched = vh_pos_first_mapping.loc[~vh_pos_first_mapping.direction.isna()]
json_mismatched = vh_pos_first_mapping.loc[vh_pos_first_mapping.direction.isna()]

In [31]:
# details of the initial outcome (after first mapping)

print(f'Size of original vh_pos data                                     : {len(vh_pos)}')
print(f'Size of successfully matched vh_pos data after join (shp & gtfs) : {len(json_matched)}')
print(f'Initial Rows lost (after initial mapping)                        : {len(vh_pos) - len(json_matched)}')

Size of original vh_pos data                                     : 19421883
Size of successfully matched vh_pos data after join (shp & gtfs) : 14499075
Initial Rows lost (after initial mapping)                        : 4922808


In [32]:
# function to preprocess the json combined data

def preprocess_json_combined_data(df):
    df['vh_date'] = df['vh_timestamp'].dt.date
    df['vh_time'] = df['vh_timestamp'].dt.time
    df['vh_time'] = df['vh_time'].astype('str').str[:8]

    df = df.drop(
        columns = [
            'stop_id',
        ]
    ).rename(
        columns = {
            'stop_seq': 'prev_stop_seq',
            'stop_name': 'prev_stop_name'
        }
    )[[
        'vh_timestamp', 
        'vh_date', 
        'vh_time', 
        'vehicle', 
        'line', 
        'direction', 
        'prev_stop_id', 
        'prev_stop_name', 
        'prev_stop_seq', 
        'distance_from_prev_stop', 
        'end_stop_id', 
        'end_stop_name'
    ]]
    
    return df

In [33]:
# apply preprocessing for both matched and mismatched data

json_matched = preprocess_json_combined_data(json_matched.copy())
json_mismatched = preprocess_json_combined_data(json_mismatched.copy())

In [34]:
# convert the NaN values to None (for mismatched data), since postgres only recognises None as null values

json_mismatched[['vehicle', 'direction', 'prev_stop_name', 'prev_stop_seq', 'end_stop_name']] = None

## 6. Load Data into DB & Perform Second Mapping for Mismatched Cases

#### Note: Second mapping involves dealing with the cases where the vehicle does not stop at the last stop for their respective line 

In [35]:
# drop tables if exists in db
cur.execute(
    """
    
    DROP TABLE IF EXISTS stop_details_combined;
    DROP TABLE IF EXISTS vh_pos;
    DROP TABLE IF EXISTS json_matched;
    DROP TABLE IF EXISTS json_mismatched;
    
    
    """
)
print("SQL Status Output:\n", cur.statusmessage)

SQL Status Output:
 DROP TABLE


In [36]:
# create tables for db
cur.execute(
    """

    -- create table for storing stop_details that were combined from both shp and gtfs data
    
    create table stop_details_combined
    (
        vehicle varchar(10),
        line varchar(10),
        direction integer,
        stop_seq integer,
        stop_id integer,
        stop_name varchar(50),
        end_stop_id integer,
        end_stop_name varchar(50)
    );
    
    -- create table for final vh_pos data
    
    create table vh_pos
    (
        vh_timestamp timestamp,
        vh_date date,
        vh_time time,
        vehicle varchar(10),
        line varchar(10),
        direction integer,
        prev_stop_id integer,
        prev_stop_name varchar(50),
        prev_stop_seq integer,
        distance_from_prev_stop numeric(7, 2),
        end_stop_id integer,
        end_stop_name varchar(50)
    );
    
    -- create temp table for json data that matched with stop_details
    
    create table json_matched
    (
        vh_timestamp timestamp,
        vh_date date,
        vh_time time,
        vehicle varchar(10),
        line varchar(10),
        direction integer,
        prev_stop_id integer,
        prev_stop_name varchar(50),
        prev_stop_seq integer,
        distance_from_prev_stop numeric(7, 2),
        end_stop_id integer,
        end_stop_name varchar(50)
    );
    
    -- create temp table for json data that mismatched with stop_details
    
    create table json_mismatched
    (
        vh_timestamp timestamp,
        vh_date date,
        vh_time time,
        vehicle varchar(10),
        line varchar(10),
        direction integer,
        prev_stop_id integer,
        prev_stop_name varchar(50),
        prev_stop_seq integer,
        distance_from_prev_stop numeric(7, 2),
        end_stop_id integer,
        end_stop_name varchar(50)
    );
    
    
    """
)
print("SQL Status Output:\n", cur.statusmessage)

SQL Status Output:
 CREATE TABLE


In [37]:
# function to insert the data from dataframe to db table for each case

def insert_df_into_db_table(df, table_name):
    print(f'*** Populating DB table {table_name}, {len(df)} tuples ***')
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s" % (table_name, cols)
    try:
        extras.execute_values(cur, query, tuples)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Insert Error: %s" % error)
    else:
        print(f'DB table {table_name} has been populated\n')

In [38]:
# apply function to insert the data from dataframe to db table

insert_df_into_db_table(stop_details, 'stop_details_combined')
insert_df_into_db_table(json_matched, 'json_matched')
insert_df_into_db_table(json_mismatched, 'json_mismatched')

*** Populating DB table stop_details_combined, 7963 tuples ***
DB table stop_details_combined has been populated

*** Populating DB table json_matched, 14499075 tuples ***
DB table json_matched has been populated

*** Populating DB table json_mismatched, 4922808 tuples ***
DB table json_mismatched has been populated



In [39]:
# query to deal with the mismatched data, then insert both the original matched data and fixed mismatch cases into one final table vh_pos

cur.execute(
    """

    insert into vh_pos

        -- get all unique lines, direction, stop_seq, stop_id

        with base as (
            select
              distinct
              vehicle
            , line
            , direction
            , stop_seq
            , stop_id
            , stop_name
            from stop_details_combined
        )

        -- for each stop (in each specific line & direction), get all previous stops that were passed in order to reach that stop (as new rows)

        , passed_stops as (
            select
              b1.*
            , b2.stop_id as passed_stop_id
            , b2.stop_name as passed_stop_name
            , b2.stop_seq as passed_stop_seq
            from base b1
            left join base b2
                on b1.line = b2.line
                    and b1.direction = b2.direction
                    and b1.stop_seq > b2.stop_seq
        )

        -- complete the integration to fix the cases that were mismathed

        , fixed_mismatched_cases as (
            select
              jm.vh_timestamp
            , jm.vh_date
            , jm.vh_time
            , ps.vehicle
            , jm.line
            , ps.direction
            , jm.prev_stop_id
            , ps.passed_stop_name as prev_stop_name
            , ps.passed_stop_seq as prev_stop_seq
            , jm.distance_from_prev_stop
            , jm.end_stop_id
            , ps.stop_name as end_stop_name
            from json_mismatched jm
            inner join passed_stops ps
                on jm.line = ps.line
                    and jm.end_stop_id = ps.stop_id
                    and jm.prev_stop_id = ps.passed_stop_id
        )

        -- union both the original matched cases, and the new fixed cases together for inserting into vh_pos table

        select * from (
            select * from json_matched
            union all
            select * from fixed_mismatched_cases
        ) x
        order by
              vh_timestamp
            , line
            , direction
            , prev_stop_seq
    
    
    """
)
print("SQL Status Output:\n", cur.statusmessage)

SQL Status Output:
 INSERT 0 16264088


In [40]:
# drop the two temp tables as they are not needed anymore

cur.execute(
    """
    
    DROP TABLE IF EXISTS json_matched;
    DROP TABLE IF EXISTS json_mismatched;
    
    """
)
print("SQL Status Output:\n", cur.statusmessage)

SQL Status Output:
 DROP TABLE


In [41]:
# get the total size of the final vh_pos table

cur.execute("select count(*) from vh_pos;")

final_vh_pos_size = pd.DataFrame(cur.fetchall(), columns = [desc[0] for desc in cur.description]).values.tolist()[0][0]

In [42]:
# details of the final outcome

print(f'Size of original vh_pos data                      : {len(vh_pos)}')
print(f'Size of vh_pos after two layer mappings completed : {final_vh_pos_size}')
print(f'Rows lost (final)                                 : {len(vh_pos) - final_vh_pos_size}')

Size of original vh_pos data                      : 19421883
Size of vh_pos after two layer mappings completed : 16264088
Rows lost (final)                                 : 3157795


In [43]:
# close cursor connection to db

cur.close()

## End.